<a href="https://colab.research.google.com/github/mingyeeee/RUhacks2021/blob/main/RUhacks.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install -q tflite-model-maker

In [ ]:
# Enable TensorFlow 2.x
%tensorflow_version 2.x

In [ ]:
import os

import numpy as np

import tensorflow as tf
assert tf.__version__.startswith('2')

from tflite_model_maker import configs
from tflite_model_maker import ExportFormat
from tflite_model_maker import image_classifier
from tflite_model_maker import ImageClassifierDataLoader
from tflite_model_maker import model_spec

import matplotlib.pyplot as plt

In [ ]:
image_path = "/content/drive/MyDrive/physics_lab"
data = ImageClassifierDataLoader.from_folder(image_path)

In [ ]:
train_data, rest_data = data.split(0.8)
validation_data, test_data = rest_data.split(0.5)

In [ ]:
mobilenet_v2_spec = model_spec.ImageModelSpec(uri='https://tfhub.dev/google/imagenet/mobilenet_v2_100_96/classification/4')
mobilenet_v2_spec.input_image_shape = [96,96]

model = image_classifier.create(train_data, model_spec=mobilenet_v2_spec, validation_data=validation_data)

In [ ]:
loss, accuracy = model.evaluate(test_data)

In [ ]:
# A helper function that returns 'red'/'black' depending on if its two input
# parameter matches or not.
def get_label_color(val1, val2):
  if val1 == val2:
    return 'black'
  else:
    return 'red'

# Then plot 100 test images and their predicted labels.
# If a prediction result is different from the label provided label in "test"
# dataset, we will highlight it in red color.
plt.figure(figsize=(20, 20))
predicts = model.predict_top_k(test_data)
for i, (image, label) in enumerate(test_data.gen_dataset().unbatch().take(100)):
  ax = plt.subplot(10, 10, i+1)
  plt.xticks([])
  plt.yticks([])
  plt.grid(False)
  plt.imshow(image.numpy(), cmap=plt.cm.gray)

  predict_label = predicts[i][0][0]
  color = get_label_color(predict_label,
                          test_data.index_to_label[label.numpy()])
  ax.xaxis.label.set_color(color)
  plt.xlabel('Predicted: %s' % predict_label)
plt.show()


In [ ]:
# Create tflite model 
model.export(export_dir='/content/drive/MyDrive/RUHACKS', export_format=ExportFormat.SAVED_MODEL)

In [ ]:
# labels file
model.export(export_dir='/content', export_format=ExportFormat.LABEL)